In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Reading test data

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 81kB 3.8MB/s 


In [ ]:
from keras.models import load_model
from pickle import load

In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor, export_graphviz

import matplotlib.pyplot as plt

import graphviz
import math



In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Final_Hack/test.csv')
location=pd.read_csv('/content/drive/MyDrive/Final_Hack/location_details.csv')
item_id=pd.read_csv('/content/drive/MyDrive/Final_Hack/item_details.csv')
transaction_train=pd.read_csv('/content/drive/MyDrive/Final_Hack/train_transactions.csv')

### Loading pickle files for the trained models

In [1]:
# model = load_model('/content/drive/MyDrive/Final_Hack/Pickle_ANN_Model_withTargeEncoder_Alldata.h5')
# encoder_model1 = load(open('/content/drive/MyDrive/Final_Hack/c1.pkl', 'rb'))
# encoder_model2 = load(open('/content/drive/MyDrive/Final_Hack/c2.pkl', 'rb'))
# encoder_model3 = load(open('/content/drive/MyDrive/Final_Hack/c3.pkl', 'rb'))

#loaded_model = pickle.load(open("/content/drive/MyDrive/Final_Hack/Pickle_XGB_Model_Alldata.pkl", "rb"))

# Preprocessing Test Data

In [ ]:
test_merged=pd.merge(test,location,left_on='locationId',right_on='location_id',how='left')
test_merged=pd.merge(test_merged,item_id,left_on='item_id',right_on='item_id',how='left')

In [ ]:
test_merged.drop('location_id',axis=1,inplace=True)

In [ ]:
test_merged['date']=pd.to_datetime(test_merged['date'])

In [ ]:
test_merged['dayofweek']=test_merged['date'].dt.dayofweek
test_merged['month']=test_merged['date'].dt.month

In [ ]:
#test_merged['dayofweek']=test_merged['dayofweek'].astype('category')
test_merged['month']=test_merged['month'].astype('category')

In [ ]:
test_merged.drop('date', axis=1, inplace=True)
#test_merged.drop('item_id', axis=1, inplace=True)
test_merged.drop('class', axis=1, inplace=True)

test_merged.drop('onpromotion', axis=1, inplace=True)
test_merged.drop('type', axis=1, inplace=True)
test_merged.drop('city', axis=1, inplace=True)
test_merged.drop('state', axis=1, inplace=True)


test_merged.drop('id',axis=1,inplace=True)


In [ ]:
test_merged['category_of_item']=test_merged['category_of_item'].astype('category')
test_merged['cluster']=test_merged['cluster'].astype('category')
test_merged['perishable']=test_merged['perishable'].astype('category')
test_merged['locationId']=test_merged['locationId'].astype('category')

In [ ]:
test_merged.drop('dayofweek', axis=1, inplace=True)

KeyError: ignored

In [ ]:
test_merged['item_id']=test_merged['item_id'].astype('category')

In [ ]:
#test_merged.drop(['cluster','perishable'],axis=1,inplace=True)
test_merged.head()

,id,date,locationId,item_id,onpromotion,city,state,type,cluster,category_of_item,class,perishable,dayofweek,month
0,0,2019-01-01,location_25,item_99197,False,Kanpur,Uttar Pradesh,D,1,grocery_items,class_1067,0,1,1
1,1,2019-01-01,location_25,item_103665,False,Kanpur,Uttar Pradesh,D,1,baked_items / bread_based,class_2712,1,1,1
2,2,2019-01-01,location_25,item_105574,False,Kanpur,Uttar Pradesh,D,1,grocery_items,class_1045,0,1,1
3,3,2019-01-01,location_25,item_105857,False,Kanpur,Uttar Pradesh,D,1,grocery_items,class_1092,0,1,1
4,4,2019-01-01,location_25,item_106716,False,Kanpur,Uttar Pradesh,D,1,grocery_items,class_1032,0,1,1


### Converting categorical cols to numeric using dummification

In [ ]:
cols_to_transform1 = test_merged.select_dtypes(include=['category','uint8']).columns
test_merged = pd.get_dummies(columns=cols_to_transform1, data=test_merged, prefix=cols_to_transform1, prefix_sep="_",drop_first=True)

In [ ]:
test_data=encoder_model.transform(test_merged)

In [ ]:
df=pd.DataFrame(np.zeros((23517680,5)),columns=list('abcde'))

## Target Encoding for all variables

In [ ]:
class_1=test_merged[['locationId','category_of_item','month','item_id']]
class_2=test_merged[['onpromotion','city','state']]
class_3=test_merged[['type','class','dayofweek']]

In [ ]:
test_merged['dayofweek']=test_merged['dayofweek'].astype('category')
test_merged['class']=test_merged['class'].astype('category')
test_merged['type']=test_merged['type'].astype('category')
test_merged['city']=test_merged['city'].astype('category')
test_merged['state']=test_merged['state'].astype('category')

In [ ]:
class_1.dtypes

locationId          category
category_of_item    category
month               category
item_id             category
dtype: object

In [ ]:
t1=encoder_model1.transform(class_1)
t2=encoder_model2.transform(class_2)
t3=encoder_model3.transform(class_3)

In [ ]:
test_data=pd.concat([t1,t2,t3],axis=1)

In [ ]:
test_data['onpromotion']=test_data['onpromotion'].astype('int')
test_data.head()

,locationId,category_of_item,month,item_id,onpromotion,city,state,type,class,dayofweek
0,3.919831,4.970956,5.133624,3.598750,0,3.919831,4.233805,4.857485,2.801564,5.558041
1,3.919831,5.793065,5.133624,4.438550,0,3.919831,4.233805,4.857485,4.663193,5.558041
2,3.919831,4.970956,5.133624,8.483656,0,3.919831,4.233805,4.857485,6.517628,5.558041
3,3.919831,4.970956,5.133624,5.224587,0,3.919831,4.233805,4.857485,3.814413,5.558041
4,3.919831,4.970956,5.133624,5.762258,0,3.919831,4.233805,4.857485,4.307675,5.558041


In [ ]:
df['a']=df['a'].astype('int8')
df['b']=df['b'].astype('int8')
df['c']=df['c'].astype('int8')
df['d']=df['d'].astype('int8')
df['e']=df['e'].astype('int8')

In [ ]:
test_m=pd.concat([test_merged,df],axis=1)

In [ ]:
test_m.drop('e',axis=1,inplace=True)

In [ ]:
x_test=test_data.values

In [ ]:
x_test.shape

(23517680, 5)

# sales forecast using predict function for trained Model

In [ ]:
y_pred = model.predict(test_data)



In [ ]:
result=pd.DataFrame(y_pred,columns=['unit_sales'])

In [ ]:
result['id']=test['id']

In [ ]:
result.head()

,unit_sales,id
0,1.999582,0
1,2.204405,1
2,6.419739,2
3,2.854342,3
4,3.159754,4


In [ ]:
result.set_index('id',inplace=True)

In [ ]:
result['unit_sales']=result['unit_sales'].astype('float32')

In [ ]:
result.to_csv('/content/drive/MyDrive/Final_Hack/submission_ANN_TargetEncoder_AllData.csv')